In [56]:
#import chatterbot
import os
import json
import matplotlib.pyplot as plt
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import spacy
from spacy import displacy
from collections import Counter
nltk.download('punkt')
import pandas as pd
from nltk import word_tokenize
from nltk.stem.snowball import SnowballStemmer
import requests
from bs4 import BeautifulSoup
from LeXmo import LeXmo
import random

[nltk_data] Downloading package punkt to /Users/amit/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [57]:
def fix_text(text):
    text = text.replace("â\x80\x99", "'")
    if "ð" in text:
        for word in text.split():
            if "ð" in word:
                text = text.replace(word, "EMOJI")
    return text


#This chunk of code reads the json files for FACEBOOK MESSENGER MESSAGES
person = "Amit" #Which person do you want the chat bot to replicate?
sender = "Amit Krishnaiyer"
path = "../Messages/" + person
conversation_list = [] #List of all conversations in an organized format
speaker_list = [] #List of everything the speaker said
for filename in os.listdir(path):
    if filename[-4:] == "json":
        conversation = []
        f = open(path + "/" + filename)
        data = json.load(f)["messages"]
        for message in data:
            if "content" in message:
                the_message = fix_text(message["content"])
                conversation.append((the_message, message["sender_name"]))
                if sender == message["sender_name"]:
                    speaker_list.append(the_message)
        conversation_list.append(conversation)


In [59]:
#This block of code gets the speaker's most common used words
word_lis = []
for message in speaker_list:
    message = preprocesser(message, True, False, False, True, True)# (text, lower, punctuation, named_entity, white_space, stop_words)
    if message.strip() != "amit  played,":
        words = message.split()
        for word in words:
            if word != "":
                word_lis.append(word)

# Pass the split_it list to instance of Counter class.

Counter = Counter(word_lis)
  
# most_common() produces k frequently encountered
# input values and their respective counts.
most_occur = Counter.most_common(30)
  
print(most_occur)
        
    
    

[('u', 111), ('t', 82), ('yea', 80), ('ur', 74), ('cuz', 66), ('like', 65), ('lol', 53), ('get', 52), ("i'm", 49), ('n', 47), ('also', 42), ('lke', 42), ('thnk', 38), ('rlly', 38), ('w', 37), ('ab', 29), ('emoji', 28), ('go', 28), ('well', 27), ('tho', 27), ("'m", 26), ('good', 26), ('f', 26), ('would', 25), ('one', 25), ('got', 24), ('see', 24), ('even', 24), ('wanna', 23), ("that's", 23)]


In [58]:
'''
Only have to run the code below once
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')
'''

lemmatizer = WordNetLemmatizer()

stop_words_set = stopwords.words('english')
custom_stop_words = ["um", "uhm", "uh", "oh", "ok", "turn!"]
stop_words_set = set(stop_words_set + custom_stop_words)



def find_and_replace(sentence): #Named Entity Recognition, replace words with general categories
    dic = {}
    for sent in nltk.sent_tokenize(sentence):
        for chunk in nltk.ne_chunk(nltk.pos_tag(nltk.word_tokenize(sent))):
             if hasattr(chunk, 'label'):
                #print(f"{' '.join(c[0] for c in chunk):<35} {chunk.label()}")
                thing = ' '.join(c[0] for c in chunk)
                dic[thing] = chunk.label()
                #print(thing, chunk.label())
    for key in dic:
        sentence = sentence.replace(key, dic[key])
    return sentence

def remove_punctuation(word): #Removes the puntuation at the end of the word
    if word[0].isalpha() == False and word[0].isdigit() == False:
        word = word[1:]
    if len(word) > 0:
        if word[-1].isalpha() == False and word[-1].isdigit() == False:
            word = word[:-1]
    return word

def replace_text_slang(text):
    URL = "https://www.slicktext.com/blog/2019/02/text-abbreviations-guide/"
    page = requests.get(URL)
    soup = BeautifulSoup(page.content, "html.parser")
    results = soup.find("ol")
    acronyms = results.find_all("li")
    dic = {}
    for i in range(len(acronyms)):
        lis = acronyms[i].text.split(":")
        lis[0] = lis[0].lower().strip()
        lis[1] = lis[1].lower().strip()
        dic[lis[0]] = lis[1]
    for key in dic:
        text.replace(key, dic[key])
    return text

def preprocesser(text, lower, punctuation, named_entity, white_space, stop_words):
    if lower:
        text = text.lower()
    if punctuation:
        text = remove_punctuation(text)
    if named_entity:
        text = find_and_replace(text)
    if white_space:
        text = text.strip()
    if stop_words:
        for word in text.split():
            if word in stop_words_set:
                text = text.replace(word, "")    
    return text

  
sentence = "As President and CEO of TribalVision, Rahul brings over 15 years of global business experience within client services having worked and lived in New York, Switzerland, Beijing, and California. Rahul is a mission-driven leader with experience in building organizations and leadership teams with more focus, more growth and more enjoyment. He brings extensive experience working effectively in high-growth environments to set a clear vision and strategy and build strong systems, processes, and infrastructure necessary to sustainably scale businesses to their maximum potential. He brings a highly client-focused mindset and makes it a core part of the company’s goals and values."

print(find_and_replace(sentence))

As President and ORGANIZATION of ORGANIZATION, GPE brings over 15 years of global business experience within client services having worked and lived in GPE, GPE, GPE, and GPE. GPE is a mission-driven leader with experience in building organizations and leadership teams with more focus, more growth and more enjoyment. He brings extensive experience working effectively in high-growth environments to set a clear vision and strategy and build strong systems, processes, and infrastructure necessary to sustainably scale businesses to their maximum potential. He brings a highly client-focused mindset and makes it a core part of the company’s goals and values.


In [55]:
from chatterbot import ChatBot
chatbot = ChatBot(person)

preprocessors=['chatterbot.preprocessors.clean_whitespace']

[nltk_data] Downloading package stopwords to /Users/amit/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/amit/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [56]:
from chatterbot.trainers import ListTrainer

trainer = ListTrainer(chatbot)

trainer.train([
    "Hi there!",
    "Hello!!",
])

trainer.train([
    "Greetings!",
    "Hello!",
])
response = chatbot.get_response("Greetings!")
print(response)


'''
What do I consider when responding to texts?
High Priority:
Energy of the message: I try to match energy: Could measure energy of message?
What is the sentiment of the message?
What are my interests at the moment?
Who are the people I'm talking about the most at the time. 


Bonus: What are they asking for- is it something I can google?


Low Priority:
What time of day is it? Do I text differently at night than day? 

'''

List Trainer: [####################] 100%
List Trainer: [####################] 100%
Hello!


"\nWhat do I consider when responding to texts?\nHigh Priority:\nEnergy of the message: I try to match energy: Could measure energy of message?\nWhat is the sentiment of the message?\nWhat are my interests at the moment?\nWho are the people I'm talking about the most at the time. \n\n\nBonus: What are they asking for- is it something I can google?\n\n\nLow Priority:\nWhat time of day is it? Do I text differently at night than day? \n\n"

In [25]:
#Emotional Classification: https://devblogs.microsoft.com/cse/2015/11/29/emotion-detection-and-recognition-from-text-using-deep-learning/
#Five categories: Happy, Angry, Sad, Surprise, Fear
t = random.choice(speaker_list)
emo=LeXmo.LeXmo(t)
print(emo)


{'text': 'U r by being good at reading', 'anger': 0.0, 'anticipation': 0.14285714285714285, 'disgust': 0.0, 'fear': 0.0, 'joy': 0.14285714285714285, 'negative': 0.0, 'positive': 0.14285714285714285, 'sadness': 0.0, 'surprise': 0.14285714285714285, 'trust': 0.14285714285714285}


In [14]:
from keras.models import Sequential
from keras.layers import Dense, Activation
import numpy as np
import requests
from bs4 import BeautifulSoup

r = requests.get("https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv")
soup = BeautifulSoup(r.content, 'html.parser')
inputs = []
outputs = []
#print(soup.text)
for line in soup.text.split("\n"):
    count = 0
    temp_in = []
    for num in line.split(","):
        count += 1
        if count < 9:
            temp_in.append(float(num))
        else:
            outputs.append(float(num))
    inputs.append(temp_in)

        




In [13]:
model = Sequential()
model.add(Dense(12, input_shape=(8,), activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(inputs, outputs, epochs=150, batch_size=10)